In [13]:
# importando bibliotecas utilizadas
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from tensorflow.keras.models import load_model

# carregando o dataset CIFAR-10
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# normalizando os valores dos pixels entre 0 e 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# definindo as categorias do CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [14]:
# construindo uma simples CNN
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

# compilando o modelo
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# treinando o modelo
model.fit(train_images, train_labels, epochs=5,
          validation_data=(test_images, test_labels))

# salvando o modelo treinado
model.save('cifar10_cnn_model.h5')

Epoch 1/5


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-bf335a30bd8b>", line 13, in <cell line: 13>
    uvicorn.run(app, port=8000)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 575, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/u

1563/1563 [==============================] - 68s 42ms/step - loss: 1.5040 - accuracy: 0.4520 - val_loss: 1.2183 - val_accuracy: 0.5571
Epoch 2/5
1563/1563 [==============================] - 73s 47ms/step - loss: 1.1295 - accuracy: 0.5988 - val_loss: 1.0702 - val_accuracy: 0.6192
Epoch 3/5
1563/1563 [==============================] - 76s 49ms/step - loss: 0.9871 - accuracy: 0.6501 - val_loss: 1.0070 - val_accuracy: 0.6441
Epoch 4/5
1563/1563 [==============================] - 76s 48ms/step - loss: 0.8887 - accuracy: 0.6890 - val_loss: 0.9567 - val_accuracy: 0.6648
Epoch 5/5
1563/1563 [==============================] - 69s 44ms/step - loss: 0.8115 - accuracy: 0.7173 - val_loss: 0.8807 - val_accuracy: 0.6930


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
# carregando o modelo criado acima
model = load_model('cifar10_cnn_model.h5')

# utilizando fast api para criar o endpoint
app = FastAPI()

# endpoint que recebe uma imagem como input e retorna o tipo dessa imagem
@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        image_bytes = await file.read()

        img = tf.image.decode_image(image_bytes, channels=3)
        img = tf.image.resize(img, [32, 32])
        img = np.expand_dims(img, axis=0)

        predictions = model.predict(img)
        predicted_class = np.argmax(predictions[0])

        return JSONResponse({'Classe predita': class_names[predicted_class]})
    except Exception as e:
        return JSONResponse({'Error': str(e)}, status_code=500)

In [16]:
# fazendo autenticação no ngrok
!ngrok authtoken 2UXGkKkItdNf01EzsmYTh3lJogJ_xFutvmbTo8yyKecZ5MDv

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# conectando e iniciando o servidor
ngrok_tunnel = ngrok.connect(8000)

print("Public URL:", ngrok_tunnel.public_url)

nest_asyncio.apply()

uvicorn.run(app, port=8000)

Public URL: https://5c18-34-16-150-16.ngrok-free.app


INFO:     Started server process [262]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


1/1 [==============================] - 0s 128ms/step
INFO:     177.92.77.170:0 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 21ms/step
INFO:     177.92.77.170:0 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 37ms/step
INFO:     177.92.77.170:0 - "POST /predict HTTP/1.1" 200 OK
